## Import require library

In [1]:
!pip install dash
!pip install jupyter_dash

In [2]:
import pandas as pd
import dash
# import dash_html_components as html
# import dash_core_components as dcc
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
# from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
from dash import no_update

### Create a dash application

In [3]:
app = dash.Dash(__name__)
# JupyterDash.infer_jupyter_proxy_config()
# app.config.suppress_callback_exceptions = True

load the dataset of spaceX

In [4]:
spacex = pd.read_csv('spacex_launch_dash.csv')


In [5]:
spacex.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


### Application layout

In [6]:
# Calculate the maximum and minimum payload mass. 
max_payload = spacex['Payload Mass (kg)'].max()
min_payload = spacex['Payload Mass (kg)'].min()

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', 
                                                 options=[{'label': 'All Sites',     'value': 'ALL'},
                                                          {'label': 'CCAFS LC-40',   'value': 'CCAFS LC-40'},
                                                          {'label': 'CCAFS SLC-40',  'value': 'CCAFS SLC-40'},
                                                          {'label': 'KSC LC-39A',    'value': 'KSC LC-39A'},
                                                          {'label': 'VAFB SLC-4E',   'value': 'VAFB SLC-4E'},
                                                          ],
                                             value ='ALL',
                                             searchable=True,
                                             placeholder='Select a Launch Site here',
                                             style={'width': '80%', 'padding': '3px','font-size': '20px', 'text-align-last': 'center'}),
                                html.Br(),
                                #Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider', 
                                                min=0, 
                                                max= 10000,
                                                step=1000, 
                                                value= [min_payload, max_payload]),
                               
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [ ]:
# add callback decorator
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))

def get_graph(site_dropdown):
    if site_dropdown == 'ALL':
        piechart = px.pie(spacex,values='class', names='Launch Site', title=f"Success Launches for site {site_dropdown}") 
        return piechart

    else:
        df= spacex[spacex['Launch Site'] == site_dropdown]
        df= df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        piechart = px.pie(df, values='class count', names='class', title=f"Success Launches for site {site_dropdown}" )
        return piechart

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'),
                 Input(component_id="payload-slider", component_property='value'))

def get_scatter(site_dropdown,slider_range,):
    low, high = slider_range
    mask = (spacex['Payload Mass (kg)'] > low) & (spacex['Payload Mass (kg)'] < high)
    df = spacex[mask]
    
    if site_dropdown == 'ALL':
        
        fig = px.scatter(df,x="Payload Mass (kg)", y="class", color="Booster Version Category", title=f'Payload vs. Outcome for site {site_dropdown}')
        return fig
    else:
        df_1= df[df['Launch Site'] == site_dropdown]
        fig = px.scatter(df_1, x="Payload Mass (kg)", y="class", color="Booster Version Category", title=f"Payload and Booster Versions for site {site_dropdown}")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Nov/2021 13:41:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 13:41:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 13:41:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 13:41:24] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 13:41:24] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 13:41:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 13:41:25] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 13:41:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 13:41:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 13:42:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2021 13:42:10] "POST /_dash-